In [2]:
import os, sys
import random, h5py
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.recurrent import LSTM
from keras.layers import Dropout
from keras.layers import Activation
from keras.optimizers import SGD

#Initialize the Program
alphabet = "NACGT."
vocab_size = 6
batch_size = 1000
embedding_size = 4
time_steps = 101
category = 2
char_to_int = dict((c, i) for i, c in enumerate(alphabet))
int_to_char = dict((i, c) for i, c in enumerate(alphabet))


Using TensorFlow backend.


TypeError: __init__() got an unexpected keyword argument 'serialized_options'

In [1]:
import tensorflow as tf

TypeError: __init__() got an unexpected keyword argument 'serialized_options'

In [2]:
def reverse_complement(sequence):
    retseq = ''
    for k in range(len(sequence)-1, -1, -1):
        if sequence[k] == 'A':
            retseq = retseq + 'T'
        elif sequence[k] == 'T':
            retseq = retseq + 'A'
        elif sequence[k] == 'C':
            retseq = retseq + 'G'
        elif sequence[k] == 'G':
            retseq = retseq + 'C'
        else:
            retseq = retseq + sequence[k]

    """
    print("Revese:")
    print(sequence)
    print(retseq)
    print()
    """
    return retseq


In [10]:
def preprocess_data(file_name, label):
    content_data = []
    count = 0
    limit = 1000
    with open(file_name) as fp:
        for line in fp:
            line_content = []
            for c in line.strip():
                line_content.append(int(c))
            content_data.append(line_content)
            count += 1
            if count > limit:
                break
    content_label = [label]*count
    return content_data, content_label

In [11]:
postive_data, positive_label = preprocess_data("/mnt/scratch4/hirak/deep_annotator_data/positve_sample.txt", 1)
neg_data, neg_label = preprocess_data("/mnt/scratch4/hirak/deep_annotator_data/negative_sample.txt", 0)

In [20]:
def one_hot_encode(sequence, n_unique):
    encoding = np.zeros((len(sequence), n_unique))
    
    for i in range(len(sequence)):
        encoding[i][sequence[i]] = 1
    
    return encoding

#print(max(source_data[0]), max(target_data[0]))
#print(one_hot_encode(source_data[0], vocab_size))
#print(one_hot_encode(target_data[0], vocab_size))

In [11]:
source_encoded = np.array([one_hot_encode(seq, vocab_size-1) for seq in source_data[:]])
print(source_encoded.shape)
    

(200000, 526, 5)


In [12]:
source_encoded[0]

array([[0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       ...,
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.]])

In [13]:
target_encoded = np.array([one_hot_encode(seq, vocab_size-1) for seq in target_data[:]])
print(target_encoded.shape)
    

(200000, 526, 5)


In [14]:
target_encoded[0]

array([[0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       ...,
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.]])

In [15]:
classes

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [12]:
merged_content = postive_data + neg_data
merged_label = positive_label + neg_label

In [22]:
encoded = np.array([one_hot_encode(seq, 5) for seq in merged_content[:]])
encoded.shape

(2002, 101, 5)

AttributeError: module 'numpy' has no attribute 'shuffle'

# Baseline Model

In [41]:
# define the encoder-decoder model
def baseline_model(n_timesteps_in, n_features, n_classes):
    model = Sequential()
    model.add(LSTM(150, input_shape=(n_timesteps_in, n_features), return_sequences=True))
    model.add(LSTM(150, return_sequences=False))
    #model.add(Flatten(input_shape=(n_timesteps_in, n_features)))
    
    model.add(Dropout(0.25))
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.25))
    
    model.add(Dense(n_classes, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
    model.summary()
    return model


In [43]:
from random import randint
from numpy import array
from numpy import argmax
from numpy import array_equal
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import RepeatVector
#from custom_recurrents import AttentionDecoder
#from keras_tqdm import TQDMNotebookCallback

print('Train...')

train_data_size = 2000
model = baseline_model(len(encoded[0]), 5, 1)

model.fit(encoded[:train_data_size], \
          merged_label[:train_data_size], \
          validation_split = 0.3, \
          shuffle = True,
          batch_size=10, epochs=3, verbose=2)

Train...


TypeError: while_loop() got an unexpected keyword argument 'maximum_iterations'

# Attention Layer

In [19]:
MAX_LEN = max_source_len # maximum sequence length
DIM_ENC = 5 # dimension of a one-hot encoded vector (e.g., 4 (sequence) x 4 (structure) = 16)
DIM_LSTM1 = 16
DIM_LSTM2 = 16
DIM_DENSE1 = 256
DMI_DENSE2 = 128
N_CLASSES = len(classes[0])

In [20]:
from keras.layers import Input
from keras.layers import Embedding
from keras.layers import Masking
from keras.layers import Reshape, Dense
from keras.models import Model
from deepMiRGene import SoftAttention

# define the encoder-decoder with attention model
def attention_model(MAX_LEN, N_CLASSES, DIM_ENC=5, \
                    DIM_LSTM1=16, DIM_LSTM2=16, \
                    DIM_DENSE1=256, DMI_DENSE2=128):
    
    inputs = Input(shape=(MAX_LEN,DIM_ENC), name='inputs')
    msk = Masking(mask_value=0)(inputs)
    lstm1 = LSTM(DIM_LSTM1, return_sequences=True, dropout=0.1, recurrent_dropout=0.1)(msk)
    lstm2 = LSTM(DIM_LSTM2, return_sequences=True, dropout=0.1, recurrent_dropout=0.1)(lstm1)

    att, pv = SoftAttention(lstm2)(lstm2)

    do1 = Dropout(0.1)(att)
    dense1 = Dense(DIM_DENSE1,activation='sigmoid')(do1)
    do2 = Dropout(0.1)(dense1)
    dense2 = Dense(DMI_DENSE2,activation='sigmoid')(do2)
    outputs = Dense(N_CLASSES,activation='softmax')(dense2)

    model=Model(outputs=outputs, inputs=inputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
    model.summary()
    return model




In [21]:
from random import randint
from numpy import array
from numpy import argmax
from numpy import array_equal
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import RepeatVector
from custom_recurrents import AttentionDecoder
from keras_tqdm import TQDMNotebookCallback

print('Train...')

train_data_size = 100000
test_source_len = max_source_len
model = attention_model(MAX_LEN, N_CLASSES)

model.fit(target_encoded[:train_data_size, :test_source_len, :], \
          classes[:train_data_size], callbacks=[TQDMNotebookCallback()], \
          validation_data=(source_encoded[train_data_size:, :test_source_len, :], \
          classes[train_data_size:]), batch_size=100, epochs=10, verbose=2)

Train...
Mask:  Tensor("masking_1/Any_1:0", shape=(?, 526), dtype=bool)
Cast:  Tensor("soft_attention_1/Cast:0", shape=(?, 526), dtype=float32)
Return PT
Vector:  Tensor("soft_attention_1/mul_1:0", shape=(?, 526), dtype=float32)
Vector:  Tensor("soft_attention_1/ExpandDims:0", shape=(?, 526, 1), dtype=float32)
Object:  Tensor("lstm_2/transpose_2:0", shape=(?, ?, 16), dtype=float32) (None, 526, 16)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 526, 5)            0         
_________________________________________________________________
masking_1 (Masking)          (None, 526, 5)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 526, 16)           1408      
_________________________________________________________________
lstm_2 (LSTM)                (None, 526, 16)           2112      
_____

In [22]:
from random import randint
from numpy import array
from numpy import argmax
from numpy import array_equal
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import RepeatVector
from custom_recurrents import AttentionDecoder
from keras_tqdm import TQDMNotebookCallback

print('Train...')

train_data_size = 100000
test_source_len = max_source_len
nodel = attention_model(MAX_LEN, N_CLASSES)

nodel.fit(target_encoded[:train_data_size, :test_source_len, :], \
          classes[:train_data_size], callbacks=[TQDMNotebookCallback()], \
          validation_data=(source_encoded[train_data_size:, :test_source_len, :], \
          classes[train_data_size:]), batch_size=100, epochs=10, verbose=2)

Train...
Mask:  Tensor("masking_2/Any_1:0", shape=(?, 526), dtype=bool)
Cast:  Tensor("soft_attention_2/Cast:0", shape=(?, 526), dtype=float32)
Return PT
Vector:  Tensor("soft_attention_2/mul_1:0", shape=(?, 526), dtype=float32)
Vector:  Tensor("soft_attention_2/ExpandDims:0", shape=(?, 526, 1), dtype=float32)
Object:  Tensor("lstm_4/transpose_2:0", shape=(?, ?, 16), dtype=float32) (None, 526, 16)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 526, 5)            0         
_________________________________________________________________
masking_2 (Masking)          (None, 526, 5)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 526, 16)           1408      
_________________________________________________________________
lstm_4 (LSTM)                (None, 526, 16)           2112      
_____